# Deploy Scikit Learn model to a Serverless Endpoint

## Setup environment

In [ ]:
import yaml
import json
import boto3
import mlflow
import logging
import sagemaker
import mlflow.sagemaker
from sklearn import datasets

logging.basicConfig(level=logging.INFO)

# IAM ROLE
iam_role = sagemaker.get_execution_role()

# AWS REGION
region = sagemaker.Session().boto_region_name

# CONFIG
with open("../cfg/model_deploy.yaml") as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

# SET MLFLOW TRACKING URI
mlflow.set_tracking_uri(config["model"]["tracking_uri"])

## Create Model

In [ ]:
# MLFLOW MODEL URI
model_uri = f"models:/{config['model']['name']}/{config['model']['version']}"

## Deploy

In [ ]:
# DEPLOY ENDPOINT
logging.info("DEPLOYING ENDPOINT")
mlflow.sagemaker.deploy(
    mode="create",
    app_name=config["endpoint"]["name"],
    model_uri=model_uri,
    image_url=config["endpoint"]["image_uri"],
    execution_role_arn=iam_role,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    region_name=region,
)


## Predict

In [ ]:
# load housing dataset
df = datasets.fetch_california_housing(as_frame=True).frame
df = df.drop("MedHouseVal", axis=1)

In [ ]:
runtime= boto3.client('runtime.sagemaker')

# predict on the first row of the dataset
payload = df.iloc[[0]].to_json(orient="split")

runtime_response = runtime.invoke_endpoint(
    EndpointName=config["endpoint"]["name"], 
    ContentType='application/json', 
    Body=payload
)
result = json.loads(runtime_response['Body'].read().decode())

print(f'Payload: {payload} \n')
print(f'Prediction: {result}')

## Delete endpoint

In [ ]:
mlflow.sagemaker.delete(app_name=config["endpoint"]["name"], region_name=region)